### Prerequisites

To run this .NET Interactive run book, you need a create an AppRegistration in Azure AD with the following settings
- Authentication -> Platform: Mobile and Desktop
- Authentication -> Platform -> RedirectUris: Msal...
- Authentication -> Supported Account Types: Single Tenant
- Authentication -> Advanced Settings -> Allow public client flows: True
- Press **Save**

- API Permissions -> Add a Permission -> Delegated Permisions -> MS Graph: `Application.ReadWrite.All`

We start by importing the necessary NuGet packages to be used throughout this runbook

In [ ]:
#r "nuget:Microsoft.Graph,3.32.0"
#r "nuget:Microsoft.Identity.Client,4.31.0"
#r "nuget:Microsoft.Graph.Auth,1.0.0-preview.7"

### Create the MSAL Client
Ensure that you add the appropriate `usings` so that we can interact with Graph and AAD.

Instantiate the MSAL Client to authenticate against AAD and get the right Graph Permissions.
The MSAL Client requires the 2 following properties:
- `ClientID` (From the Overview tab of your AAD App Registration)
- `TenantID` (From the Overview tab. Can be the tenant id [Guid] or the tenant name)

In [ ]:
using Microsoft.Identity.Client;
using System.Threading.Tasks;
using Microsoft.Graph.Auth;

public static DeviceCodeProvider GetMsalClient()
{
    var clientId = "c369fdb7-d70b-4651-9e5c-4379c3863d78";
    var tenantId = "b55f0c51-61a7-45c3-84df-33569b247796";
    
    IPublicClientApplication publicClientApplication = PublicClientApplicationBuilder
        .Create(clientId)
        .WithTenantId(tenantId)
        .Build();
    var scopes = new string[]{"Application.ReadWrite.All", "User.Read"};
    return new DeviceCodeProvider(publicClientApplication, scopes);
}

### Create an API App Registration in Azure AD

Since we will be authenticating a daemon/headless app with no user interaction
we can't use scopes. In Azure AD, we use Application-only app roles for authorization.
The following code snippet creates an API App Registration in AAD along with the appropriate
app roles.
Right now the app role is hardcoded to `access_as_application`. 
Feel free to change this to what is appropriate for your app.

In [ ]:
using Microsoft.Graph;

var apiAppName = "Python Auto Demo API";

var graphClient = new GraphServiceClient(GetMsalClient());
var apiApp = GetApiApplication(apiAppName);

var apiAppRegistration = await graphClient.Applications
                            .Request()
                            .AddAsync(apiApp);

private async Task<string> GetTenantId()
{
    var organization = await graphClient.Organization
                                .Request()
                                .GetAsync();
    return organization.FirstOrDefault().Id;
}

var tenantId = await GetTenantId();

Console.WriteLine($"Client Id: {apiAppRegistration.AppId}");
Console.WriteLine($"Domain: {apiAppRegistration.PublisherDomain}");
Console.WriteLine($"Tenant Id: {tenantId}");

private static Application GetApiApplication(string displayName) => new Application
{
    DisplayName = displayName,
    IdentifierUris = new List<String>()
    {
        $"api://{Guid.NewGuid().ToString()}"
    },
    Api = new ApiApplication
    {
        AcceptMappedClaims = null,
        KnownClientApplications = new List<Guid>()
        {
        },
        RequestedAccessTokenVersion = 2,
        PreAuthorizedApplications = new List<PreAuthorizedApplication>()
        {
        }
    },
    AppRoles = new List<AppRole> 
    {
        new AppRole
        {
            Id = Guid.NewGuid(),
            DisplayName = "Access as application",
            Description = "App Role for client application",
            IsEnabled = true,
            Origin = "Application",
            Value = "access_as_application",
            AllowedMemberTypes = new [] {"Application"}
        }
    }  
};

### Create the Service Principal
Next we need to create a Service Principal that maps to the App Registration. 
This registers the app with the current Tenant and makes the App available to 
the rest of the tenant's apps and users to authenticate against and/or acquire 
tokens.

In [ ]:
using Microsoft.Graph;

var servicePrincipal = await graphClient.ServicePrincipals
                        .Request()
                        .AddAsync(CreateServicePrincipal(apiAppRegistration.AppId));

private static ServicePrincipal CreateServicePrincipal(string appId) => new ServicePrincipal
{    
    AppId = appId
};

### Console/Deamon App App Registration [Optional]
Create a client App Registration for a console app that signs in users interactively (.NET, Node, Java etc).

Users signing in to the client app, will be able to request an access token for the API ().
You'll need to set the following two properties before running this script:
- `clientAppName` -> the name for your App Registration
- `redirectUri` -> the URI where your app expects the returned tokens. For desktop apps you can use the MSAL one or a custom such as: 
`http://localhost`

In [ ]:
using Microsoft.Graph;

var clientAppName = "Python auto Demo Console";

var clientApp = GetClientApplication(
    apiAppRegistration,
    clientAppName);

var consoleAppRegistration = await graphClient.Applications
                                .Request()
                                .AddAsync(clientApp);

Console.WriteLine($"Client Id: {consoleAppRegistration.AppId}");
Console.WriteLine($"Domain: {consoleAppRegistration.PublisherDomain}");
Console.WriteLine($"Tenant Id: {tenantId}");
                           
private static Application GetClientApplication(Application app,string displayName) 
    => new Application
{
    DisplayName = displayName,
    RequiredResourceAccess = new List<RequiredResourceAccess>()
    {
        new RequiredResourceAccess
        {
            ResourceAppId = app.AppId,
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {
                    Id = app.AppRoles.FirstOrDefault().Id,
                    Type = "Role"
                }
            }
        },
        new RequiredResourceAccess
        {
            // OIDC-Graph scope (user.read)
            ResourceAppId = "00000003-0000-0000-c000-000000000000",
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {     
                    Id = new Guid("e1fe6dd8ba314d6189e788639da4683d"),
                    Type = "Scope"
                }
            }
        }
    } 
};

### Add a client secret

In [ ]:
using Microsoft.Graph;

var passwordCredential = new PasswordCredential
{
	DisplayName = "delete me"
};

var credential = await graphClient.Applications[$"{consoleAppRegistration.Id}"]
	.AddPassword(passwordCredential)
	.Request()
	.PostAsync();

Console.WriteLine($"Client Secret: {credential.SecretText}");